In [1]:
# new_session
from fastapi import FastAPI, Request
import dependencies.orion_utils as orion
import dependencies.geojson_utils as geojs
import dependencies.ngsi_utils as ngsi
import paho.mqtt.client as Paho

import modules.iot_platform.cosmicswamp as cosmic
import modules.simulation_platform.pcse_simulator as pcsesm
import modules.information_platform as infplm
import modules.iot_platform as iotplm
import sys
import datetime
import time

import numpy as np


In [2]:
sr = orion.session(servicepath="/new_session")
# cosmic.delete_session(sr, purge=True)
cosmic.get_all_current_entity_ids(sr)


[DEBUG:cosmicswamp:24-09-24T12:41:32.7]: http://backend-orion:1026/v2/entities?attrs=id,type&limit=100 {'Fiware-Servicepath': '/new_session', 'Fiware-Service': 'openiot'}


[{'id': 'urn:ngsi-ld:Field:1', 'type': 'Field'},
 {'id': 'urn:ngsi-ld:Field:1:ManagementZone:0-0', 'type': 'ManagementZone'},
 {'id': 'urn:ngsi-ld:Field:1:ManagementZone:0-1', 'type': 'ManagementZone'},
 {'id': 'urn:ngsi-ld:Field:1:ManagementZone:0-2', 'type': 'ManagementZone'},
 {'id': 'urn:ngsi-ld:Field:1:ManagementZone:1-0', 'type': 'ManagementZone'},
 {'id': 'urn:ngsi-ld:Field:1:ManagementZone:1-1', 'type': 'ManagementZone'},
 {'id': 'urn:ngsi-ld:Field:1:ManagementZone:1-2', 'type': 'ManagementZone'},
 {'id': 'urn:ngsi-ld:Field:1:ManagementZone:2-0', 'type': 'ManagementZone'},
 {'id': 'urn:ngsi-ld:Field:1:ManagementZone:2-1', 'type': 'ManagementZone'},
 {'id': 'urn:ngsi-ld:Field:1:ManagementZone:2-2', 'type': 'ManagementZone'},
 {'id': 'urn:ngsi-ld:SoilProbe:F1-0-0', 'type': 'SoilDepthProbe'},
 {'id': 'urn:ngsi-ld:SoilProbe:F1-0-1', 'type': 'SoilDepthProbe'},
 {'id': 'urn:ngsi-ld:SoilProbe:F1-0-2', 'type': 'SoilDepthProbe'},
 {'id': 'urn:ngsi-ld:SoilProbe:F1-1-0', 'type': 'SoilDept

In [3]:
# import dependencies.leaflet_utils as leaf
# import ipyleaflet

# m = leaf.build_map([-45.523917847717236,-12.170341419421398])
# e = cosmic.get_current_entity_from_id(sr, "Field:1")

# def reverse_latlon_pairs(latlon_pairs):
#     """
#     Reverses a list of (latitude, longitude) pairs to (longitude, latitude) pairs.
    
#     Parameters:
#     latlon_pairs (list of tuples): List of (latitude, longitude) pairs.
    
#     Returns:
#     list of tuples: List of (longitude, latitude) pairs.
#     """
#     return [(lon, lat) for lat, lon in latlon_pairs]
    
# polygon = ipyleaflet.Polygon(
#     locations=reverse_latlon_pairs(e["location"]['value']['coordinates'][0]),
#     color="green",
#     fill_color="green"
# )

# m.add(polygon)
# leaf.display_map(m)


In [4]:

# def field_series(session, entity_id, *args, **kwargs):
#     """Proxy for cosmic.series(session, id, args, kwargs)."""
#     return cosmic.get_series_data_for_entity(session, entity_id, *args, **kwargs)

# def field_series(session, entity_id, *args, **kwargs):
#     """Proxy for cosmic.series(session, id, args, kwargs)."""
#     return cosmic.get_series_data_for_entity(session, entity_id, *args, **kwargs)


infplm.field.series = cosmic.get_series_data_for_entity
infplm.field.state  = cosmic.estimate_entity_state_at_time

# infplm.soildepthprobe.payload = infplm.soildepthprobe.iotagent
# infplm.soildepthprobe.series = cosmic.get_series_data_for_entity
# infplm.soildepthprobe.state  = cosmic.estimate_entity_state_at_time
# infplm.soildepthprobe.forward  = infplm.soildepthprobe.calibrate_data_forward
# infplm.soildepthprobe.backward  = infplm.soildepthprobe.calibrate_data_backward

# Field Creation
infplm.field.create(sr,
                    "Field:1",
                    location=field_extent)

# Alternative Pivot Creation
infplm.pivotfield.create(sr, 
    "Field:1", 
    center_location=gjs.Point([-45.523917847717236,-12.170341419421398]), 
    radius=600*m)

# Example zones
infplm.pivotfield.create(sr,
                    "Field:1",
                    location=field_extent,
                    radius=600*m)
infplm.pivotfield.split_into_radial_zones(6, 6)

for zone in infplm.pivotfield.get_zones("Field:1"):    
    zone_location = zone["location"]
    zone_id = zone["tag"]
    
    infplm.soildepthprobe.create(sr,
                                 f"Probe:{zone_id}",
                                 zone_location)
                            
    
    




In [5]:
# seriesd = infplm.field.series(sr, "Field:1", time_index=[1326690811000,1726690811000])
seriesd = infplm.field.series(sr, "Field:1", time_index=[1326690811000,2726690811000])
stated  = infplm.field.state(sr,  "Field:1", time_index=1726690811000)

seriesd = infplm.soildepthprobe.series(sr, "SoilDepthProbe:1", time_index=[1326690811000,2726690811000])



[DEBUG:cosmicswamp:24-09-23T20:52:56.2]: SELECTION SELECT time_index, timeinstant, location, zones  FROM "mtopeniot"."etfield" WHERE fiware_servicepath='/new_session'  AND entity_id='urn:ngsi-ld:Field:1'   AND time_index > 1326690811000 AND time_index < 2726690811000   ORDER BY time_index ASC LIMIT 1000
RETRIEVING SELECT time_index, timeinstant, location, zones  FROM "mtopeniot"."etfield" WHERE fiware_servicepath='/new_session'  AND entity_id='urn:ngsi-ld:Field:1'   AND time_index > 1326690811000 AND time_index < 2726690811000   ORDER BY time_index ASC LIMIT 1000 crate://backend-cratedb:4200
RETRIEVING SELECT entity_id, *, abs(time_index - 1726690811000.0) as dif, time_index as timeinstant, time_index as uploadinstant  FROM "mtopeniot"."etfield" WHERE fiware_servicepath='/new_session'  AND entity_id='urn:ngsi-ld:Field:1'  ORDER BY ABS(time_index - 1726690811000.0) ASC LIMIT 1 crate://backend-cratedb:4200


In [6]:
print(seriesd)
import pandas as pd

pd.DataFrame(data=seriesd)


[]


""


In [7]:
print(stated)

{'id': 'urn:ngsi-ld:Field:1', 'type': 'Field', 'TimeInstant': {'type': 'DateTime', 'value': 1727124760000, 'metadata': {}}, 'location': {'type': 'geo:json', 'value': {'coordinates': [[[-45.523918, -12.164945], [-45.525347, -12.165129], [-45.526678, -12.165668], [-45.527821, -12.166525], [-45.528698, -12.167643], [-45.52925, -12.168944], [-45.529438, -12.170341], [-45.52925, -12.171738], [-45.528699, -12.173039], [-45.527821, -12.174156], [-45.526678, -12.175014], [-45.525347, -12.175553], [-45.523918, -12.175737], [-45.522489, -12.175553], [-45.521158, -12.175014], [-45.520015, -12.174156], [-45.519138, -12.173039], [-45.518586, -12.171738], [-45.518398, -12.170341], [-45.518586, -12.168944], [-45.519138, -12.167643], [-45.520015, -12.166525], [-45.521158, -12.165668], [-45.522489, -12.165129], [-45.523918, -12.164945]]], 'type': 'Polygon'}, 'metadata': {}}, 'zones': {'type': 'StructuredValue', 'value': {'id': ['Field:1:ManagementZone:0-0', 'Field:1:ManagementZone:0-1', 'Field:1:Manage

In [21]:
import dependencies.time_utils as tm

# Example simulate the smearing. 
for i in range(5):
    infplm.soildepthprobe.update(sr, 
                                 "SoilDepthProbe:1", 
                                 time_index = tm.stamp("23-09-24 12:51:15") + 1*tm.hour,
                                 soiltemperature0=i*100)


# e = cosmic.get_current_entity_from_id(sr, "SoilDepthProbe:1")
# cosmic.delete(sr, "SoilDepthProbe:1", "SoilDepthProbe", purge=True)
cosmic.create(sr, entity_id="SoilDepthProbe:1", jsondata=e)
e = cosmic.get_current_entity_from_id(sr, "SoilDepthProbe:1")

print(e)

{'id': 'urn:ngsi-ld:SoilDepthProbe:1', 'type': 'SoilDepthProbe', 'TimeInstant': {'type': 'DateTime', 'value': '2024-09-23T13:51:15.00Z', 'metadata': {}}, 'UploadInstant': {'type': 'DateTime', 'value': '2024-09-23T20:57:09.00Z', 'metadata': {}}, 'soilcalibration0': {'type': 'StructuredValue', 'value': {'function': 'a + b*x + c/x/x', 'constants': {'a': 0.25257, 'b': 0.0002503, 'c': -4581.3}}, 'metadata': {}}, 'soilcalibration1': {'type': 'StructuredValue', 'value': {'function': 'a + b*x + c/x/x', 'constants': {'a': 0.25257, 'b': 0.0002503, 'c': -4581.3}}, 'metadata': {}}, 'soilcalibration2': {'type': 'StructuredValue', 'value': {'function': 'a + b*x + c/x/x', 'constants': {'a': 0.25257, 'b': 0.0002503, 'c': -4581.3}}, 'metadata': {}}, 'soilmoisturecalibrated0': {'type': 'Number', 'value': 0, 'metadata': {'units': {'type': 'String', 'value': 'm3/m3'}}}, 'soilmoisturecalibrated1': {'type': 'Number', 'value': 0, 'metadata': {'units': {'type': 'String', 'value': 'm3/m3'}}}, 'soilmoisturecali

In [22]:
pd.DataFrame(data=cosmic.series(sr, "SoilDepthProbe:1"))


[DEBUG:cosmicswamp:24-09-23T20:57:34.7]: SELECTION SELECT time_index, timeinstant, uploadinstant, soilcalibration0, soilcalibration1, soilcalibration2, soilmoisturecalibrated0, soilmoisturecalibrated1, soilmoisturecalibrated2, soiltemperature0, source, state  FROM "mtopeniot"."etsoildepthprobe" WHERE fiware_servicepath='/new_session'  AND entity_id='urn:ngsi-ld:SoilDepthProbe:1'     ORDER BY time_index ASC LIMIT 1000
RETRIEVING SELECT time_index, timeinstant, uploadinstant, soilcalibration0, soilcalibration1, soilcalibration2, soilmoisturecalibrated0, soilmoisturecalibrated1, soilmoisturecalibrated2, soiltemperature0, source, state  FROM "mtopeniot"."etsoildepthprobe" WHERE fiware_servicepath='/new_session'  AND entity_id='urn:ngsi-ld:SoilDepthProbe:1'     ORDER BY time_index ASC LIMIT 1000 crate://backend-cratedb:4200


,time_index,timeinstant,uploadinstant,soilcalibration0,soilcalibration1,soilcalibration2,soilmoisturecalibrated0,soilmoisturecalibrated1,soilmoisturecalibrated2,soiltemperature0,source,state
0,1727099475000,1727099475000,1727125029253,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,0.0,API,SIMULATED
1,1727099475000,1727099475000,1727125029309,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,100.0,API,SIMULATED
2,1727099475000,1727099475000,1727125052844,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,100.0,API,SIMULATED
3,1727099475000,1727099475000,1727125029451,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,300.0,API,SIMULATED
4,1727099475000,1727099475000,1727125029483,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,400.0,API,SIMULATED
5,1727099475000,1727099475000,1727125029403,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,200.0,API,SIMULATED
6,1727099475000,1727099475000,1727125043929,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,300.0,API,SIMULATED
7,1727099475000,1727099475000,1727125043804,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,100.0,API,SIMULATED
8,1727099475000,1727099475000,1727125043967,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,400.0,API,SIMULATED
9,1727099475000,1727099475000,1727125043875,"{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...","{'function': 'a + b*x + c/x/x', 'constants': {...",0.0,0.0,0.0,200.0,API,SIMULATED


In [ ]:


infplm.field.create( 